### Llama-3.2-3B-Instruct (fine-tuning using unsloth)

In [70]:
pip install -r requirements.txt

### Loading the model and Tokenizer for fine-tuning

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage

model_id = "unsloth/Llama-3.2-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id ,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

### We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, 
)

Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

### Data Transformation done

In [8]:
import pandas as pd
from datasets import Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Step 2: Load CSV and prepare data in the expected format
df = pd.read_csv("final_data.csv")
data = []


system_prompt = """
You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.

For any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.

However, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:

'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'

Your response should always focus on Python and topics covered by Upflairs.
"""
for i in range(df.shape[0]):
    sample = {
        'conversations': [
            {"from": "system", "value": system_prompt},
            {'from': 'human', 'value': df.loc[i, "Question"]},
            {'from': 'gpt', 'value': df.loc[i, "Answer"]}
        ]
    }
    data.append(sample)

print("No. of samples in the dataset:", len(data))

# Step 3: Convert list of dictionaries to a Dataset object
dataset = Dataset.from_list(data)

# Step 4: Apply the standardization function
dataset = standardize_sharegpt(dataset)

# Step 5: Define the formatting function to apply the chat template
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

# Apply the formatting function with map
dataset = dataset.map(formatting_prompts_func, batched=True)

# Optional: View a sample of the formatted dataset
print(dataset[0])


No. of samples in the dataset: 39


Standardizing format:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

{'conversations': [{'content': "\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n", 'role': 'system'}, {'content': 'tell me about upflairs?', 'role': 'user'}, {'content': "UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging techn

In [ ]:
print(dataset[2])

{'conversations': [{'content': "\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n", 'role': 'system'}, {'content': 'How can I enroll in a course?', 'role': 'user'}, {'content': "You can enroll in any course by visiting the Upflairs website, selecting the course you're interested in, and clicking the 'Enro

<a name="Train"></a>
### configuring training arguments with  `SFTTrainer`

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# set the hyperparameters for supervised fine-tuning
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 15,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/39 [00:00<?, ? examples/s]

Unsloth Trainer initialaizing, so that we can fine-tune SFTTrainer with unsloth.

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

In [26]:
print("No. of Documents after tokenized : ",len(trainer.train_dataset))

No. of Documents after tokenized :  39


In [29]:
# each query has 3 keys
trainer.train_dataset[5].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

We verify masking on data:

In [27]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow long is the internship program at Upflairs?<|eot_id|><|start_header_

We can see the System and Instruction prompts are successfully masked!

In [47]:
trainer_stats = trainer.train()   # start the fine-tuning with epochs=1

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 39 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.421700
2,1.530200
3,1.122100
4,1.835000
5,1.492000
6,1.057800
7,0.985100
8,0.959700
9,0.970100
10,0.567800


In [63]:
## ask the query after the training model
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",)

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

messages = [
    {"role":"system","content":"You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.Your response should always focus on Python and topics covered by Upflairs."},
    {"role": "user", "content": "tell me about upflairs?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1, min_p = 0.1)
tokenizer.batch_decode(outputs)   # output with masking

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.Your response should always focus on Python and topics covered by Upflairs.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me about upflairs?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nUp

In [64]:
# Feteched Data from masked output
response = tokenizer.batch_decode(outputs)
print("Robo response : ",response[0].split('\n')[-1]) # extracted exact response from the chat response


Robo response :  UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. Our mission is to educate 1 million students worldwide by 2025 and provide


In [66]:
# Define a list of testing queries (we can also add system prompt for that follow prevous one  inferencing)
test_queries = [
    {"role": "user", "content": "tell me about upflairs."},
    {"role": "user", "content": "What courses does Upflairs offer?"},
    {"role": "user", "content": "How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?"},
    {"role": "user", "content": "write a program to print hello world."},
    {"role": "user", "content": "write a program to print hello world in python."},
    {"role": "user", "content": "write a program to print hello world in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in c++?"},
    {"role": "user", "content": "how we can find the average of the integer array in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in python?"}
]

# Process each query using the chat template and tokenize
for query in test_queries:
    inputs = tokenizer.apply_chat_template(
        [query],
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate responses with varied parameters for testing
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=64,
        use_cache=True,
        temperature=0.7,  # Lowered temperature for more deterministic responses
        min_p=0.1
    )

    # Decode and print each output
    response = tokenizer.batch_decode(outputs)
    print(f"Query: {query['content']}")
    print("Robo response : ",response[0].split('\n')[-1][:-10]) # extracted exact response from the chat response
    print()


Query: tell me about upflairs.
Robo response :  Upflairs is a UK-based online retailer that specializes in selling beauty and health products, including skincare, makeup, and wellness items. They offer a wide range of products from various brands, and often have sales and discounts on certain items. Upflairs also has a loyalty program that rewards customers for repeat purchases.

Query: What courses does Upflairs offer?
Robo response :  UpFlairs offers courses in AI/ML, Web Development, Data Science, Cloud Computing, Cyber Security, and more.

Query: How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?
Robo response :  seen_numbers = [num for num in range(1, 101) if num % 2 == 0]

Query: write a program to print hello world.
Robo response :  print('Hello World')

Query: write a program to print hello world in python.
Robo response :  print('Hello World')

Query: write a program to print hello world in java?
Robo response :  }

Query: how we can 

In [67]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [68]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How do you write a Python function at Upflairs to check if a string is a palindrome?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

def is_palindrome(s):
	s = s.lower()
	return s == s[::-1]
print(is_palindrome('UpFlairs'))<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

### Thank You